In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

# Ignora i warning
import warnings
warnings.filterwarnings('ignore')

# Carica il dataset
dataset_link_prediction = pd.read_csv('machine-global-merged.csv')

# Estrai i generi unici
generi = set()
with open('machine-global-merged.csv', 'r') as f:
    csv_reader = csv.reader(f)
    for line in csv_reader:
        if len(line) > 3:
            s = line[3].replace("[","").replace("]","").replace("'","")
            genres = s.split(",")
            for genre in genres:
                generi.add(genre.strip())

# Aggiungi colonne per ogni genere al dataframe
for genere in generi:
    dataset_link_prediction[genere] = 0

# Popola il dataframe con i generi
riga = -1
with open('machine-global-merged.csv', 'r') as f:
    csv_reader = csv.reader(f)
    for line in csv_reader:
        if line[0] != 'paese':
            riga += 1
            if len(line) > 3:
                s = line[3].replace("[","").replace("]","").replace("'","")
                genres = s.split(",")
                for genre in genres:
                    dataset_link_prediction.loc[riga, genre.strip()] = 1

# Seleziona le features e le etichette
X = dataset_link_prediction.iloc[:, 4:]  # Matrice delle features
y = dataset_link_prediction['paese']  # Etichette

# Suddividi i dati in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Trasforma le stringhe in valori numerici (se possibile)
X_train_numeric = X_train.apply(pd.to_numeric, errors='coerce').fillna(0)

# Applica la trasformazione MinMaxScaler alle features di training
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train_numeric)

# Imputa i valori mancanti nelle features di training
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_minmax)

# Rimuovi righe con etichette mancanti dai dati di addestramento
y_train_cleaned = y_train.dropna()

# Rimuovi le righe corrispondenti dalle features di addestramento
X_train_imputed_cleaned = X_train_imputed[:len(y_train_cleaned)]

# Crea e addestra il classificatore Logistic Regression
clf_logreg = LogisticRegression()
clf_logreg.fit(X_train_imputed_cleaned, y_train_cleaned)

# Applica la trasformazione MinMaxScaler ai dati di test
X_test_numeric = X_test.apply(pd.to_numeric, errors='coerce').fillna(0)
X_test_minmax = min_max_scaler.transform(X_test_numeric)

# Imputa i valori mancanti nelle features di test
X_test_imputed = imputer.transform(X_test_minmax)

# Predici i risultati sui dati di test
y_predicted_lr = clf_logreg.predict(X_test_imputed)

# Valuta le prestazioni del modello
print(classification_report(y_test.astype(str), y_predicted_lr.astype(str)))
print(accuracy_score(y_test.astype(str), y_predicted_lr.astype(str)))


                                                                                                                                                                                                                           precision    recall  f1-score   support

                                                                                                                                                                                                                       ;;       0.09      1.00      0.17        58
                                                                                                                                                                                                           Arabia_Saudita       1.00      0.05      0.10        20
                                                                   Arabia_Saudita,GBKPL2205058,spotify:track:3qhlB30KknSejmIvZZLjOD,"['pov: indie', 'psychedelic pop']",0.689,0.454,0.0584,0.0351,0.00259,0.912,159.982;;     